# Importing MMR dataset in Neo4j
Possible optimizations:
- Multiple relationships are added between users: why not adding info on such as the period? After encoding it.
- I could remove the username string from the nodes to save space, but let's experiment with the performance before.
- Total Relationships: 612497446 (~600M)
- Total nodes: 89486651 (~90M)